In [25]:
# imports

# data science
import pandas as pd

# langchain
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.chains import SimpleSequentialChain
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import ChatPromptTemplate
from langchain.prompts import PromptTemplate

# langchain-ollama
from langchain_ollama import ChatOllama

In [5]:
# read data
df = pd.read_csv('Data.csv')

In [6]:
df.head()

,Product,Review
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld\n,I loved this product. But they only seem to l...


In [7]:
# define llm
llm = ChatOllama(
    model="llama3.1",
    temperature=0.9,      # very stochastic
)

In [8]:
# define prompt
prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

In [9]:
# define chain
chain = LLMChain(llm=llm, prompt=prompt)

/var/folders/z5/s8fr1m057mj7x5qy6x65x9qw0000gn/T/ipykernel_52108/3255069063.py:2: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=prompt)


In [10]:
product = "Queen Size Sheet Set"
chain.run(product)

/var/folders/z5/s8fr1m057mj7x5qy6x65x9qw0000gn/T/ipykernel_52108/550859008.py:2: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  chain.run(product)


"Here are some suggestions for a company name that makes queen size sheet sets:\n\n1. **DreamWoven**: This name suggests the idea of creating comfortable, restful bedding.\n2. **QueenBare Essentials**: A playful name that highlights the queen size aspect and implies a focus on the essentials (bedding).\n3. **CozyCrown**: This name evokes a sense of warmth and comfort, while also referencing the royal feel of a queen-sized bed.\n4. **SoftSpreads**: A simple, descriptive name that emphasizes the softness and quality of their bedding.\n5. **SlumberCraft Co.**: This name conveys a focus on craftsmanship and attention to detail, which can be appealing to customers looking for high-quality bedding.\n6. **Restful Bedding Co.**: This name clearly communicates the company's mission and implies a focus on helping people get a good night's rest.\n7. **Threaded Dreams**: Another playful name that suggests a connection between quality materials (threads) and the idea of creating comfortable, dreamy

## SimpleSequentialChain

In [12]:
# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

# chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)

In [13]:
# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    "Write a 20 words description for the following \
    company:{company_name}"
)
# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [14]:
# define simple chain
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                             verbose=True
                                            )

In [15]:
overall_simple_chain.run(product)



> Entering new SimpleSequentialChain chain...
Here are some suggestions for a company name that makes Queen-size sheet sets:

1. **Regal Bedding**: This name plays off the idea of a queen-size bed being regal and luxurious.
2. **DreamWeaver Bed Linens**: This name evokes a sense of comfort and sleep, suggesting that their products will help customers create their own dreamlike experiences.
3. **Queen Supreme Bedding Co.**: This name confidently asserts the company's focus on high-quality queen-size sheet sets.
4. **Restful Nights Bedding**: This name conveys a promise of peaceful slumber and comfort, which is perfect for a bedding company.
5. **Royal Slumber Sheets**: This name combines the idea of luxury with the concept of restful sleep, implying that their products will treat customers like royalty.
6. **Slumber Queen Bedding**: This name incorporates the popular "queen" theme while emphasizing the importance of high-quality bedding for a restful night's sleep.
7. **Bedroom Bliss 

'Here is a 20-word description for the company:\n\n"Experience luxurious comfort with Regal Bedding\'s high-quality queen-size sheet sets, crafted for restful nights and unforgettable slumber, every time."'

## SequentialChain

In [17]:
# prompt template 1: translate to english
first_prompt = ChatPromptTemplate.from_template(
    "Translate the following review to english:"
    "\n\n{Review}"
)
# chain 1: input= Review and output= English_Review
chain_one = LLMChain(llm=llm, prompt=first_prompt, 
                     output_key="English_Review"      # you define an output key
                    )

In [18]:
# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    "Can you summarize the following review in 1 sentence:"
    "\n\n{English_Review}"
)
# chain 2: input= English_Review and output= summary
chain_two = LLMChain(llm=llm, prompt=second_prompt, 
                     output_key="summary"
                    )


In [19]:
# prompt template 3: translate to english
third_prompt = ChatPromptTemplate.from_template(
    "What language is the following review:\n\n{Review}"
)
# chain 3: input= Review and output= language
chain_three = LLMChain(llm=llm, prompt=third_prompt,
                       output_key="language"
                      )


In [20]:

# prompt template 4: follow up message
fourth_prompt = ChatPromptTemplate.from_template(
    "Write a follow up response to the following "
    "summary in the specified language:"
    "\n\nSummary: {summary}\n\nLanguage: {language}"
)
# chain 4: input= summary, language and output= followup_message
chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
                      output_key="followup_message"
                     )


In [21]:
# overall_chain: input= Review 
# and output= English_Review,summary, followup_message
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["Review"],
    output_variables=["English_Review", "summary","followup_message"],
    verbose=True
)

In [22]:
review = df.Review[5]
overall_chain(review)

/var/folders/z5/s8fr1m057mj7x5qy6x65x9qw0000gn/T/ipykernel_52108/1992003631.py:2: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  overall_chain(review)




> Entering new SequentialChain chain...

> Finished chain.


{'Review': "Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...\nVieux lot ou contrefaçon !?",
 'English_Review': 'Here is the translation of the review in English:\n\n"I find the taste mediocre. The mousse doesn\'t hold up, that\'s weird. I buy these same ones from the store and the taste is much better...\n\nOld stock or counterfeit?"',
 'summary': 'The reviewer found the product to be merely average in taste and was disappointed compared to other purchases of the same item.',
 'followup_message': 'Here\'s a potential follow-up response written in French:\n\n"Je suis en accord avec vous, le produit était bien loin d\'être exceptionnel comme je l\'avais espéré. Le goût était vraiment moyen et ne m\'a pas laissé un souvenir agréable. C\'est dommage car j\'étais impatient de savourer ce que j\'avais entendu être une excellente qualité. La prochaine fois, je pense que je rechercherai ailleurs."\n\nTranslat

## Router Chain

In [23]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""

In [24]:
prompt_infos = [
    {
        "name": "physics", 
        "description": "Good for answering questions about physics", 
        "prompt_template": physics_template
    },
    {
        "name": "math", 
        "description": "Good for answering math questions", 
        "prompt_template": math_template
    },
    {
        "name": "History", 
        "description": "Good for answering history questions", 
        "prompt_template": history_template
    },
    {
        "name": "computer science", 
        "description": "Good for answering computer science questions", 
        "prompt_template": computerscience_template
    }
]

In [26]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain  
    
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [27]:
# in router mode, you need to define a default
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [28]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

<>:1: SyntaxWarning: invalid escape sequence '\ '
<>:1: SyntaxWarning: invalid escape sequence '\ '
/var/folders/z5/s8fr1m057mj7x5qy6x65x9qw0000gn/T/ipykernel_52108/2160379643.py:1: SyntaxWarning: invalid escape sequence '\ '
  MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \


In [29]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [30]:
chain = MultiPromptChain(router_chain=router_chain, 
                         destination_chains=destination_chains, 
                         default_chain=default_chain, verbose=True
                        )

/var/folders/z5/s8fr1m057mj7x5qy6x65x9qw0000gn/T/ipykernel_52108/3038952769.py:1: LangChainDeprecationWarning: Use RunnableLambda to select from multiple prompt templates. See example in API reference: https://api.python.langchain.com/en/latest/chains/langchain.chains.router.multi_prompt.MultiPromptChain.html
  chain = MultiPromptChain(router_chain=router_chain,


In [31]:
chain.run("What is black body radiation?")



> Entering new MultiPromptChain chain...
physics: {'input': 'What is black body radiation?'}
> Finished chain.


"Black-body radiation! A fundamental concept in thermodynamics and statistical mechanics. I'd be happy to explain it in simple terms.\n\nA black body is an idealized physical system that absorbs all electromagnetic radiation that falls on it, without reflecting or transmitting any of it. Think of it like a perfect absorber of light and heat.\n\nBlack-body radiation refers to the thermal radiation emitted by this hypothetical object at different temperatures. When you heat up a black body (or an actual object acting as one), it starts emitting radiation across all wavelengths, from low-energy radio waves to high-energy gamma rays.\n\nThe key characteristic of black-body radiation is that its energy distribution follows a specific pattern, known as the Planck spectrum or black-body radiation curve. This curve shows how the intensity of emitted radiation varies with wavelength at different temperatures.\n\nIn essence, black-body radiation is a theoretical model that helps us understand ho

In [32]:
chain.run("what is 2 + 2")



> Entering new MultiPromptChain chain...
math: {'input': 'what is 2 + 2 ='}
> Finished chain.


'To solve this classic problem, I\'ll follow my usual approach of breaking it down into its component parts. Here\'s how I\'ll do it:\n\n**Component Part 1: Understanding what "2" means**\n\nIn mathematics, the number 2 represents a quantity that has two units or entities. Think of it like having two apples in your hand.\n\n**Component Part 2: Understanding the operation "+"**\n\nThe plus sign (+) indicates an addition operation, which means combining two quantities to get a total sum.\n\n**Component Part 3: Adding the two quantities together**\n\nNow that we understand what "2" means and what the + operation does, let\'s combine them:\n\nWe have two units of something (represented by 2). When we add another two units to it (also represented by 2), we get a total of... four units!\n\n**Putting it all together:**\n\nSo, when you ask me "what is 2 + 2?", I can confidently say that the correct answer is:\n\n2 + 2 = **4**\n\nVoilà!'

In [33]:
chain.run("Why does every cell in our body contain DNA?")



> Entering new MultiPromptChain chain...
math: {'input': 'Why does every cell contain DNA in terms of molecular structure and replication?'}
> Finished chain.


"What a fascinating question! On the surface, it seems like a biology question, but I'm excited to break it down into its mathematical components. After all, as you know, mathematics is the language of nature, and understanding the underlying principles can reveal surprising connections.\n\nTo tackle this problem, let's consider the following aspects:\n\n1. **Information Storage**: DNA (Deoxyribonucleic acid) is a molecule that contains genetic information necessary for the development, functioning, growth, and reproduction of all living organisms. Mathematically speaking, we can think of DNA as an encoding scheme, where four nucleotide bases (A, C, G, and T) are used to represent genetic information in a compact form.\n\nCan you calculate the number of possible combinations using these four bases? \n\n2. **Replication Efficiency**: The molecular structure of DNA is crucial for its replication process. Mathematically, we can analyze the probability of accurate duplication during cell d